In [1]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import pandas as pd
books=pd.read_csv('books_cleaned.csv')
books


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [4]:
from langchain.schema import Document

raw_documents = [
    Document(page_content=row)
    for row in books["tagged_description"].astype(str)
]

In [5]:
text_splitter = CharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=0,
    separator="\n"
)
documents = text_splitter.split_documents(raw_documents)

In [6]:
documents[0:5]

[Document(metadata={}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and

In [7]:
db_books=Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings()
)

In [8]:
query="A book to teach children about nature"
relevant=db_books.similarity_search(query, k=3)
relevant

[Document(id='63235e5e-2acd-49e5-982a-3ed7f50d221f', metadata={}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(id='96447360-493d-4f32-ae9d-a66c7398961f', metadata={}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
 Document(id='4e42df99-20b4-4cc3-ac1f-eaa5d5c94cb3', metadata={}, page_content="9780786808397 Introduce your baby to birds, cats, dogs, and ba

In [9]:
#now need to use this recommendation to get the actual book details from the dataframe

books[books["isbn13"]==int(relevant[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [10]:
#now converting this into a function

def retrieve_relevant_books(query, k):
    relevant=db_books.similarity_search(query, k=k)
    book_list=[]

    for i in range(0, len(relevant)):
        book=books[books["isbn13"]==int(relevant[i].page_content.split()[0].strip())]
        book_list.append(book)

    return pd.concat(book_list, ignore_index=True)

In [11]:
retrieve_relevant_books("A book about food",10)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780375758737,0375758739,Comfort Me With Apples,Ruth Reichl,Biography & Autobiography,http://books.google.com/books/content?id=keNpY...,The noted food critic describes her odyssey fr...,2002.0,4.03,302.0,17418.0,Comfort Me With Apples: More Adventures at the...,9780375758737 The noted food critic describes ...
1,9781400052370,1400052378,Eat This Book,Tyler Florence,Cooking,http://books.google.com/books/content?id=nF2Pc...,"Collects more than 150 recipes, ranging from a...",2005.0,3.92,287.0,270.0,Eat This Book: Cooking with Global Fresh Flavors,"9781400052370 Collects more than 150 recipes, ..."
2,9781580086813,1580086810,Hungry Planet,Peter Menzel;Faith D'Aluisio,Cooking,http://books.google.com/books/content?id=-fFqQ...,"""A photographic collection exploring what the ...",2005.0,4.44,288.0,1420.0,Hungry Planet: What the World Eats,"9781580086813 ""A photographic collection explo..."
3,9781400040353,1400040353,Lidia's Family Table,Lidia Matticchio Bastianich;David Nussbaum,Cooking,http://books.google.com/books/content?id=ZQNwD...,A guide to family cookery features more than t...,2004.0,4.22,448.0,521.0,Lidia's Family Table,9781400040353 A guide to family cookery featur...
4,9780972018418,0972018417,Eating for Life,Bill Phillips,Health & Fitness,http://books.google.com/books/content?id=rzqEN...,The popular fitness author explains how to ach...,2003.0,3.95,405.0,788.0,Eating for Life,9780972018418 The popular fitness author expla...
5,9781933615097,1933615095,The Best of America's Test Kitchen 2007,America's Test Kitchen,Cooking,http://books.google.com/books/content?id=tQ97A...,Presents nearly one thousand recipes--from app...,2006.0,4.34,312.0,185.0,The Best of America's Test Kitchen 2007: The Y...,9781933615097 Presents nearly one thousand rec...
6,9781400052387,1400052386,Tyler's Ultimate,Tyler Florence,Cooking,http://books.google.com/books/content?id=cGxOH...,The popular Food Network chef presents a selec...,2006.0,4.10,256.0,3586.0,Tyler's Ultimate: Brilliant Simple Food to Mak...,9781400052387 The popular Food Network chef pr...
7,9780060899226,0060899220,Kitchen Confidential Updated Ed,Anthony Bourdain,Biography & Autobiography,http://books.google.com/books/content?id=bxLbZ...,"A deliciously funny, delectably shocking banqu...",2007.0,4.06,312.0,168706.0,Kitchen Confidential Updated Ed: Adventures in...,"9780060899226 A deliciously funny, delectably ..."
8,9781568984957,1568984952,"Yours in Food, John Baldessari",John Baldessari;Paul Auster,Art,http://books.google.com/books/content?id=FtnNd...,"In John Baldessari's new book, Yours in Food, ...",2004.0,4.00,144.0,19.0,"Yours in Food, John Baldessari","9781568984957 In John Baldessari's new book, Y..."
9,9781400040360,1400040361,Lidia's Italy,Tanya Bastianich Manuali,Cooking,http://books.google.com/books/content?id=YwNwD...,The popular TV chef offers a gastronomic tour ...,2007.0,4.21,384.0,619.0,Lidia's Italy: 140 Simple and Delicious Recipe...,9781400040360 The popular TV chef offers a gas...


In [12]:
books["categories"].value_counts().reset_index().query("count>50")

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
5,Philosophy,117
6,Religion,117
7,Comics & Graphic Novels,116
8,Drama,86
9,Juvenile Nonfiction,57


In [13]:
from transformers import pipeline
from langdetect import detect
import pandas as pd

In [15]:
!pip install sacremoses


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [16]:
try:
    yo2en_translator = pipeline("translation", model="Helsinki-NLP/opus-mt-yo-en", device=-1)
except Exception as e:
    print("Yoruba->English translator failed to load:", e)
    yo2en_translator = None

Device set to use cpu


In [17]:
def translate_yoruba_to_english(text):
    """Translate Yoruba to English if translator exists; otherwise return input text."""
    if yo2en_translator is None:
        return text
    try:
        out = yo2en_translator(text, max_length=512)
        return out[0]["translation_text"]
    except Exception:
        return text

In [38]:
def retrieve_relevant_books_simple(query, k=3, translate=False):
    """
    Simple pipeline:
      - detect language (langdetect)
      - if Yoruba (or force_translate True), translate to English
      - run your existing db_books.similarity_search on the English query
      - return a single pandas.DataFrame of matched books (unique)
    Assumes:
      - `db_books` is your Chroma vector store already built over English texts
      - `books` is your original DataFrame with 'isbn13' and 'tagged_description' columns
    """
    # 1) detect language
    try:
        lang = detect(query)
    except Exception:
        lang = "en"

    if translate:
        q_for_search = translate_yoruba_to_english(query)
    else:
        q_for_search = query

    # 2) run similarity search on your existing vector store
    results = db_books.similarity_search(q_for_search, k=k)

    # 3) convert Document results into a single DataFrame (same logic as earlier)
    rows = []

    for r in results:
        content = r.page_content.strip()
        isbn_str = content.split()[0]
        matched_df = pd.DataFrame()

        if isbn_str.isdigit():
            isbn = int(isbn_str)
            matched_df = books[books["isbn13"] == isbn]

        if matched_df.empty:
            snippet = content[:120].strip()
            matched_df = books[books["tagged_description"].str.contains(snippet, na=False)]

        if not matched_df.empty:
            rows.append(matched_df)

    if not rows:
        return pd.DataFrame()

    df = pd.concat(rows, ignore_index=True)
    if "isbn13" in df.columns:
        df = df.drop_duplicates(subset=["isbn13"]).reset_index(drop=True)
    return df

In [39]:
# Yoruba-like test (you can use real Yoruba phrases)
q = "ìtàn nípa oúnjẹ"          # "story of love" (Yoruba)
print("Translated ->", translate_yoruba_to_english(q))

Translated -> Food history


In [42]:
retrieve_relevant_books_simple(q, k=10,translate=True)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780140275018,0140275010,Cod,Mark Kurlansky,History,http://books.google.com/books/content?id=6dKOl...,"A history of the fish that has led to wars, st...",1998.0,3.90,294.0,923.0,Cod: A Biography of the Fish that Changed the ...,9780140275018 A history of the fish that has l...
1,9780142001615,0142001619,Salt,Mark Kurlansky,History,http://books.google.com/books/content?id=9RdmH...,"Explores the role of salt in shaping history, ...",2003.0,3.74,484.0,46854.0,Salt: A World History,9780142001615 Explores the role of salt in sha...
2,9780802715524,0802715524,A History of the World in 6 Glasses,Tom Standage,Cooking,http://books.google.com/books/content?id=sSugA...,An offbeat history of the world traces the sto...,2006.0,3.76,336.0,13105.0,A History of the World in 6 Glasses,9780802715524 An offbeat history of the world ...
3,9780375509179,0375509178,Shakespeare's Kitchen,Francine Segan,Cooking,http://books.google.com/books/content?id=ecwAA...,A noted food historian brings to life the clas...,2003.0,3.99,288.0,108.0,Shakespeare's Kitchen: Renaissance Recipes for...,9780375509179 A noted food historian brings to...
4,9781400040353,1400040353,Lidia's Family Table,Lidia Matticchio Bastianich;David Nussbaum,Cooking,http://books.google.com/books/content?id=ZQNwD...,A guide to family cookery features more than t...,2004.0,4.22,448.0,521.0,Lidia's Family Table,9781400040353 A guide to family cookery featur...
5,9781400052370,1400052378,Eat This Book,Tyler Florence,Cooking,http://books.google.com/books/content?id=nF2Pc...,"Collects more than 150 recipes, ranging from a...",2005.0,3.92,287.0,270.0,Eat This Book: Cooking with Global Fresh Flavors,"9781400052370 Collects more than 150 recipes, ..."
6,9781580086813,1580086810,Hungry Planet,Peter Menzel;Faith D'Aluisio,Cooking,http://books.google.com/books/content?id=-fFqQ...,"""A photographic collection exploring what the ...",2005.0,4.44,288.0,1420.0,Hungry Planet: What the World Eats,"9781580086813 ""A photographic collection explo..."
7,9780060852559,0060852550,"Animal, Vegetable, Miracle",Barbara Kingsolver;Camille Kingsolver;Steven L...,Biography & Autobiography,http://books.google.com/books/content?id=qLkEY...,Bestselling author Barbara Kingsolver returns ...,2007.0,4.04,370.0,86130.0,"Animal, Vegetable, Miracle: A Year of Food Life",9780060852559 Bestselling author Barbara Kings...
8,9780375758737,0375758739,Comfort Me With Apples,Ruth Reichl,Biography & Autobiography,http://books.google.com/books/content?id=keNpY...,The noted food critic describes her odyssey fr...,2002.0,4.03,302.0,17418.0,Comfort Me With Apples: More Adventures at the...,9780375758737 The noted food critic describes ...
9,9781568984957,1568984952,"Yours in Food, John Baldessari",John Baldessari;Paul Auster,Art,http://books.google.com/books/content?id=FtnNd...,"In John Baldessari's new book, Yours in Food, ...",2004.0,4.00,144.0,19.0,"Yours in Food, John Baldessari","9781568984957 In John Baldessari's new book, Y..."
